In [61]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
from xlsxwriter.workbook import Workbook
from xlsxwriter import Workbook
import time

In [62]:
Working_dir = os.getcwd()
Data_dir = os.path.join(Working_dir, "Raw data")

In [63]:
node_df = pd.read_csv( os.path.join( Data_dir,"May2018_idle.csv"), delimiter=";", header=0 )

In [64]:
node_df.head()

,nodename,tempo,timestamp_measure,sys_power,cpu_power,mem_power,fan1a,fan1b,fan2a,fan2b,...,sys_util,cpu_util,mem_util,io_util,amb_temp,cpu1_temp,cpu2_temp,exh_temp,sysairflow,dcenergy
0,cresco6x001,Tue 15 May 16:05:48 CEST 2018,1526393148,150,110,13,3584,3712,4736,4992,...,0,0,0,0,19,45,40,38,13,"260,19502"
1,cresco6x002,Tue 15 May 16:05:58 CEST 2018,1526393148,150,110,12,3584,3712,4736,4992,...,0,0,0,0,18,45,40,38,13,"251,28987"
2,cresco6x003,Tue 15 May 16:06:07 CEST 2018,1526393148,150,110,12,3584,3712,4736,4992,...,0,0,0,0,20,47,39,40,13,"253,87433"
3,cresco6x004,Tue 15 May 16:06:16 CEST 2018,1526393148,150,110,10,3584,3712,4736,4992,...,0,0,0,0,17,44,37,37,13,"232,74909"
4,cresco6x005,Tue 15 May 16:06:26 CEST 2018,1526393148,150,110,13,3712,3712,4736,4992,...,0,0,0,0,19,43,37,38,13,"243,38527"


In [65]:
node_df.nodename.unique()

array(['cresco6x001', 'cresco6x002', 'cresco6x003', 'cresco6x004',
       'cresco6x005', 'cresco6x006', 'cresco6x007', 'cresco6x008',
       'cresco6x009', 'cresco6x010', 'cresco6x011', 'cresco6x012',
       'cresco6x013', 'cresco6x014', 'cresco6x015', 'cresco6x016',
       'cresco6x017', 'cresco6x018', 'cresco6x019', 'cresco6x020',
       'cresco6x021', 'cresco6x022', 'cresco6x023', 'cresco6x024',
       'cresco6x025', 'cresco6x026', 'cresco6x027', 'cresco6x028',
       'cresco6x029', 'cresco6x030', 'cresco6x031', 'cresco6x032',
       'cresco6x033', 'cresco6x034', 'cresco6x035', 'cresco6x036',
       'cresco6x037', 'cresco6x038', 'cresco6x039', 'cresco6x040',
       'cresco6x041', 'cresco6x042', 'cresco6x043', 'cresco6x044',
       'cresco6x045', 'cresco6x046', 'cresco6x047', 'cresco6x048',
       'cresco6x049', 'cresco6x050', 'cresco6x051', 'cresco6x052',
       'cresco6x053', 'cresco6x054', 'cresco6x055', 'cresco6x056',
       'cresco6x057', 'cresco6x058', 'cresco6x059', 'cresco6x0

In [66]:
len(node_df.nodename.unique())

215

## No node with name 'cresco6x114'

In [67]:
print(node_df[node_df.nodename == 'cresco6x114'])

Empty DataFrame
Columns: [nodename, tempo, timestamp_measure, sys_power, cpu_power, mem_power, fan1a, fan1b, fan2a, fan2b, fan3a, fan3b, fan4a, fan4b, fan5a, fan5b, sys_util, cpu_util, mem_util, io_util, amb_temp, cpu1_temp, cpu2_temp, exh_temp, sysairflow, dcenergy]
Index: []

[0 rows x 26 columns]


In [68]:
print("Compare results with search for node 113:\n\n", node_df[node_df.nodename == 'cresco6x113'][:1])

Compare results with search for node 113:

         nodename                          tempo  timestamp_measure  sys_power  \
112  cresco6x113  Tue 15 May 16:23:13 CEST 2018         1526393148        150   

     cpu_power  mem_power  fan1a  fan1b  fan2a  fan2b    ...      sys_util  \
112        110         13   4224   4352   5632   5888    ...             0   

     cpu_util  mem_util  io_util  amb_temp  cpu1_temp  cpu2_temp  exh_temp  \
112         0         0        0        19         42         39        36   

     sysairflow   dcenergy  
112          15  246,69157  

[1 rows x 26 columns]


## Check columns names

In [70]:
print(node_df.columns.values)

['nodename' 'tempo' 'timestamp_measure' 'sys_power' 'cpu_power'
 'mem_power' 'fan1a' 'fan1b' 'fan2a' 'fan2b' 'fan3a' 'fan3b' 'fan4a'
 'fan4b' 'fan5a' 'fan5b' 'sys_util' 'cpu_util' 'mem_util' 'io_util'
 'amb_temp' 'cpu1_temp' 'cpu2_temp' 'exh_temp' 'sysairflow' 'dcenergy']


## Reformat dcenergy column to have float view

In [71]:
node_df.dcenergy = node_df.dcenergy.apply(lambda x: float(x.replace(",", ".")) )

-----
# Energy used by node
- for system, CPU and memory, rectangle formula of integration is applied
- dcenergy is simply summed

In [86]:
nodes_stats = pd.DataFrame(columns = list(node_df.nodename.unique()))

In [87]:
nodes_stats.loc["total_dcenergy", :] = 0

## Calculating integral value for power using rectangle back count formula

In [88]:
for col in nodes_stats.columns.values:
    
    one_node_df = node_df[node_df.nodename == col].copy()
    row_num = one_node_df.shape[0]
    
    sys_energy = one_node_df["sys_power"][:-1].values/1000.0
    cpu_energy = one_node_df["cpu_power"][:-1].values/1000.0
    mem_energy = one_node_df["mem_power"][:-1].values/1000.0

    time_difference = (one_node_df["timestamp_measure"][1:].values -\
    one_node_df["timestamp_measure"][:-1].values)/3600.0
    
    nodes_stats.loc["sys_energy", col] = np.dot(sys_energy, time_difference)
    nodes_stats.loc["cpu_energy", col] = np.dot(cpu_energy, time_difference)
    nodes_stats.loc["mem_energy", col] = np.dot(mem_energy, time_difference)
    
    nodes_stats.loc["sys_energy_other", col] = nodes_stats.loc["sys_energy", col] - \
                                                    nodes_stats.loc["cpu_energy", col] - \
                                                    nodes_stats.loc["mem_energy", col]
            
       
    nodes_stats.loc["total_dcenergy", col] = node_df[node_df.nodename == col]["dcenergy"].max() -\
    node_df[node_df.nodename == col]["dcenergy"].min()
    
    nodes_stats.loc["total_dcenergy - sys_energy", col] = nodes_stats.loc["total_dcenergy", col] - \
                                                        nodes_stats.loc["sys_energy", col]

In [89]:
nodes_stats

,cresco6x001,cresco6x002,cresco6x003,cresco6x004,cresco6x005,cresco6x006,cresco6x007,cresco6x008,cresco6x009,cresco6x010,...,cresco6x207,cresco6x208,cresco6x209,cresco6x210,cresco6x211,cresco6x212,cresco6x213,cresco6x214,cresco6x215,cresco6x216
total_dcenergy,65.474480,75.141040,73.640830,74.992180,302.257620,75.187430,72.368760,74.068250,76.066660,75.417510,...,71.938250,73.779760,73.082720,71.171430,69.581810,72.137020,74.335280,71.541430,69.202020,56.633540
sys_energy,61.088128,69.750008,67.914197,69.645422,67.696389,69.904397,65.884961,67.683717,69.970322,70.002561,...,66.965808,68.900589,66.998642,66.948186,65.220144,66.990903,70.895481,66.987944,65.187303,58.134936
cpu_energy,45.191056,52.105000,51.956256,52.023850,51.796639,54.036467,51.654797,51.829400,54.035319,54.148503,...,51.139519,51.145156,51.291728,51.189442,51.143692,51.277922,52.492281,51.150978,51.145253,42.446433
mem_energy,5.174917,4.774370,4.764949,4.007822,5.339703,5.150401,5.535940,4.752918,3.967728,5.398535,...,5.489955,5.098126,4.706411,5.880651,5.789489,4.707522,5.820573,5.099401,5.099225,5.485126
sys_energy_other,10.722155,12.870639,11.192993,13.613750,10.560047,10.717530,8.694224,11.101398,11.967275,10.455524,...,10.336334,12.657308,11.000503,9.878093,8.286964,11.005458,12.582628,10.737566,8.942825,10.203377
total_dcenergy - sys_energy,4.386352,5.391032,5.726633,5.346758,234.561231,5.283033,6.483799,6.384533,6.096338,5.414949,...,4.972442,4.879171,6.084078,4.223244,4.361666,5.146117,3.439799,4.553486,4.014717,-1.501396


In [91]:
np.where(nodes_stats.loc["total_dcenergy - sys_energy", :] <= 0)

(array([ 46,  95, 101, 105, 162, 175, 214], dtype=int64),)

In [92]:
nodes_stats.loc[:, ['cresco6x047',  'cresco6x096', 'cresco6x102', 'cresco6x106', \
                                      'cresco6x164', 'cresco6x177', 'cresco6x216'] ]

,cresco6x047,cresco6x096,cresco6x102,cresco6x106,cresco6x164,cresco6x177,cresco6x216
total_dcenergy,11.414340,0.000000,44.044040,0.0,0.000000,48.089450,56.633540
sys_energy,63.969250,3.917519,68.662050,0.0,58.762792,66.939883,58.134936
cpu_energy,49.935697,0.000000,54.589256,0.0,47.010233,51.108633,42.446433
mem_energy,5.484527,0.000000,5.105890,0.0,4.701023,4.292883,5.485126
sys_energy_other,8.549026,3.917519,8.966904,0.0,7.051535,11.538367,10.203377
total_dcenergy - sys_energy,-52.554910,-3.917519,-24.618010,0.0,-58.762792,-18.850433,-1.501396


In [93]:
excel_writer_may = pd.ExcelWriter( os.path.join( Data_dir, "May2018_nodes_stats.xlsx"))
nodes_stats.to_excel(excel_writer_may)
excel_writer_may.save()

## Calculating integral value for power using rectangle forth count formula

In [98]:
nodes_stats = pd.DataFrame(columns = list(node_df.nodename.unique()))

In [99]:
nodes_stats.loc["total_dcenergy", :] = 0

for col in nodes_stats.columns.values:
    
    one_node_df = node_df[node_df.nodename == col].copy()
    row_num = one_node_df.shape[0]
    
    sys_energy = one_node_df["sys_power"][1:].values/1000.0
    cpu_energy = one_node_df["cpu_power"][1:].values/1000.0
    mem_energy = one_node_df["mem_power"][1:].values/1000.0

    time_difference = (one_node_df["timestamp_measure"][1:].values -\
    one_node_df["timestamp_measure"][:-1].values)/3600.0
    
    nodes_stats.loc["sys_energy", col] = np.dot(sys_energy, time_difference)
    nodes_stats.loc["cpu_energy", col] = np.dot(cpu_energy, time_difference)
    nodes_stats.loc["mem_energy", col] = np.dot(mem_energy, time_difference)
    
    nodes_stats.loc["total_dcenergy", col] = node_df[node_df.nodename == col]["dcenergy"].max() -\
    node_df[node_df.nodename == col]["dcenergy"].min()
    
nodes_stats.loc["sys_energy_other", :] = nodes_stats.loc["sys_energy", :] - \
    nodes_stats.loc["cpu_energy", :] - nodes_stats.loc["mem_energy", :]
nodes_stats.loc["total_dcenergy - sys_energy", :] = nodes_stats.loc["total_dcenergy", :] - \
    nodes_stats.loc["sys_energy", :]

In [100]:
excel_writer_may = pd.ExcelWriter( os.path.join( Data_dir, "May2018_nodes_stats_forth_integral.xlsx"))
nodes_stats.to_excel(excel_writer_may)
excel_writer_may.save()

In [101]:
nodes_stats

,cresco6x001,cresco6x002,cresco6x003,cresco6x004,cresco6x005,cresco6x006,cresco6x007,cresco6x008,cresco6x009,cresco6x010,...,cresco6x207,cresco6x208,cresco6x209,cresco6x210,cresco6x211,cresco6x212,cresco6x213,cresco6x214,cresco6x215,cresco6x216
total_dcenergy,65.474480,75.141040,73.640830,74.992180,302.257620,75.187430,72.368760,74.068250,76.066660,75.417510,...,71.938250,73.779760,73.082720,71.171430,69.581810,72.137020,74.335280,71.541430,69.202020,56.633540
sys_energy,67.553397,76.961222,75.301086,78.053600,74.551464,78.496772,74.282211,74.706258,78.552439,78.575064,...,71.683933,75.162183,71.710325,71.655969,71.492947,71.698339,75.601667,71.697517,71.456683,58.513583
cpu_energy,51.487794,59.320856,58.989817,58.856344,58.465556,62.267447,58.322772,58.673994,62.261997,62.541644,...,55.852575,55.850747,56.002711,55.893217,55.852375,55.982858,57.015389,55.854567,55.854292,43.008919
mem_energy,5.242640,4.843908,4.836218,4.075903,5.252182,5.221342,5.605061,4.823634,4.056357,5.331112,...,5.490923,5.098212,4.706672,5.880446,5.618059,4.707316,5.817641,5.098903,5.099067,5.485041
sys_energy_other,10.822963,12.796458,11.475051,15.121352,10.833726,11.007983,10.354378,11.208629,12.234085,10.702307,...,10.340435,14.213224,11.000942,9.882307,10.022514,11.008164,12.768637,10.744047,10.503324,10.019623
total_dcenergy - sys_energy,-2.078917,-1.820182,-1.660256,-3.061420,227.706156,-3.309342,-1.913451,-0.638008,-2.485779,-3.157554,...,0.254317,-1.382423,1.372395,-0.484539,-1.911137,0.438681,-1.266387,-0.156087,-2.254663,-1.880043


## Resume
- backward integral is more precise
- agreed: for columns *_util data is not available 
- 215 (!), not 216 different nodes

### Further analysis
- check dcenergy, etc columns for ['cresco6x047',  'cresco6x096', 'cresco6x102', 'cresco6x106','cresco6x164', 'cresco6x177', 'cresco6x216']
- energy correlation
- fans - energy correlation